# Analysing Strava Activity data

For a long time I am trying to connect to my strava data to do some data analysis and finally I nailed.

It took me hours and hours to learn python and various packages and finally it I did it.

I have to thank the HTPP services training I took with Talk Python to Me

https://training.talkpython.fm/courses/details/consuming-http-and-soap-services-in-python-with-json-xml-and-screen-scraping

And also for finding the StravaIO package in github that Maksym Sladkov created. It is on Github at

https://github.com/sladkovm/stravaio

and also can be intalled from pypi https://pypi.org/project/stravaio/

Still had to find a work around in a class method that was not working as planned as I could not wait for the issue to be resolved in github. So this was also kind of my first contribution to a open source project in a way

If you ever struggled to get you data from strava using python you can use the code below

After that I intend to do some EDA with the data and model the data to make predictions

The lessons learned from programming is never stop if you hit a wall because those are the momment you learn the most.


In [ ]:
from stravaio import strava_oauth2
import os
from stravaio import StravaIO
import pandas as pd
from collections import defaultdict

import swagger_client
from swagger_client.rest import ApiException


In [ ]:
#importing strava API client id and secret from local environment
strava_client_id = os.environ.get('client_id')

strava_client_secret = os.environ.get('client_secret')


In [ ]:
#authentication with Strava API
# will take to strava client and will required authorisation from athlete 

oauth_res = strava_oauth2(client_id=strava_client_id, client_secret=strava_client_secret)

In [ ]:
# store access token in variable
STRAVA_ACCESS_TOKEN = oauth_res['access_token']

In [ ]:
#instatiate stravaio constructor

client = StravaIO(access_token=STRAVA_ACCESS_TOKEN)

In [ ]:
# get logged in athlete and load into a dictionary object
athlete = client.get_logged_in_athlete()

athlete_dict = athlete.to_dict()

In [ ]:
athlete

In [ ]:
athlete_dict

In [ ]:
# time stamp to request activity in epoch time 
# use this converter to get the date: https://www.epochconverter.com/

timestamp=1420095429

In [ ]:
# Configure swagger client
access_token = STRAVA_ACCESS_TOKEN

configuration = swagger_client.Configuration()
configuration.access_token = access_token
_api_client = swagger_client.ApiClient(configuration)
athletes_api = swagger_client.AthletesApi(_api_client)
activities_api = swagger_client.ActivitiesApi(_api_client)


In [ ]:
# get request of activities

#import paginated

list_activities = []
after = timestamp
page = 1
while page <= 14:
    _fetched = activities_api.get_logged_in_athlete_activities(after=after, page=page, per_page=100)
    list_activities.extend(_fetched)
    print(f"Fetched {len(_fetched)}, between {_fetched[0].start_date} and {_fetched[-1].start_date}")
    page += 1
    if len(_fetched) <100:
        break


In [ ]:

print(f'total activities fetched were {len(list_activities)}')

In [ ]:
# transform swagger dict object into nested python dictionary

nested_dict={}
for i,d in enumerate(list_activities):
    nested_dict[i]=d.to_dict()


In [ ]:
# merge nested dictionaries into one with list of values for each key
dd = defaultdict(list)

for d in nested_dict.values():
    for k, v in d.items():
        dd[k].append(v)


In [ ]:
# create a pandas dataframe with downloaded activities

activities_df = pd.DataFrame(dd)

In [ ]:
activities_df.head()

In [ ]:
activities_df.shape

In [ ]:
#save activities_df.to_csv() for backup in the local directory

activities_df.to_csv('{}/strava_data.csv'.format(os.getcwd()))



In [ ]:
activities_df.columns

In [ ]:
activities_df.info()

In [ ]:
activities_df.describe()